In [8]:
def manh(pos_a, pos_b):
    return abs(pos_a[0] - pos_b[0]) + abs(pos_a[1] - pos_b[1])


inc_ranges = list()
beacons_of_int = list()
sensors = list()
with open("day_15_input.txt") as file:
    while line := file.readline().rstrip():
        sensor_str, beacon_str = line.split(":")
        sensor_str = [a.split("=") for a in sensor_str.split(",")]
        beacon_str = [a.split("=") for a in beacon_str.split(",")]
        sensor_cord = [int(sensor_str[0][1]), int(sensor_str[1][1])]
        beacon_cord = [int(beacon_str[0][1]), int(beacon_str[1][1])]
        sensor_range = manh(sensor_cord, beacon_cord)
        sensors.append((sensor_cord, sensor_range))
        interval_range = sensor_range - (abs(sensor_cord[1] - 2000000))
        if interval_range >= 0:
            inc_ranges.append(
                [sensor_cord[0] - interval_range, sensor_cord[0] + interval_range]
            )
            if beacon_cord[1] == 2000000 and beacon_cord[0] not in beacons_of_int:
                beacons_of_int.append(beacon_cord[0])

#### Task 1

In [9]:
# merge intervals without beacons into disjoint intervals
import copy

disjoint_ranges = list()
for i, inc_range in enumerate(sorted(copy.deepcopy(inc_ranges))):
    if i == 0:
        disjoint_ranges.append(inc_range)
        j = 0
    elif inc_range[0] <= disjoint_ranges[j][1] + 1:
        disjoint_ranges[j][1] = max(disjoint_ranges[j][1], inc_range[1])
    else:
        disjoint_ranges.append(inc_range)
        j += 1

In [10]:
# remove deacon locations and split intervals at their locations
dis_ranges_to_delete = list()
for beacon_x in beacons_of_int:
    for i, dis_range in enumerate(disjoint_ranges):
        if dis_range[0] <= beacon_x <= dis_range[1]:
            dis_ranges_to_delete.append(i)
            if beacon_x - dis_range[0] > 0:
                disjoint_ranges.append([dis_range[0], beacon_x - 1])
            if dis_range[1] - beacon_x > 0:
                disjoint_ranges.append([beacon_x + 1, dis_range[1]])
            break

In [11]:
# remove obsolete pre-deacon-removal intervals
for i in sorted(dis_ranges_to_delete, reverse=True):
    del disjoint_ranges[i]

In [12]:
disjoint_ranges

[[-651994, 652349], [652351, 5180534]]

In [13]:
sum([dis_range[1] - dis_range[0] + 1 for dis_range in disjoint_ranges])

5832528

#### Task 2

In [14]:
from scipy.optimize import linprog
from itertools import product
import numpy as np

In [15]:
# to establish the order of inequalities and thus reduce the number of considered options
sensors_by_x = sorted(sensors, key=lambda tup: tup[0][0])
sensors_by_y = sorted(sensors, key=lambda tup: tup[0][1])

In [16]:
# finding binding inequalites: // and \\ in the cartesian plane
inner_candidate_combs = list()
inner_ineq_candidate_combs = list()
for i in range(len(sensors) - 1):
    for j in range(1, len(sensors)):
        for k in range(len(sensors) - 1):
            for l in range(1, len(sensors)):
                if (
                    manh(sensors_by_x[i][0], sensors_by_x[j][0])
                    == sensors_by_x[i][1] + sensors_by_x[j][1] + 2
                ) and (
                    manh(sensors_by_y[k][0], sensors_by_y[l][0])
                    == sensors_by_y[k][1] + sensors_by_y[l][1] + 2
                ):
                    if (
                        sensors_by_x[i][0][1] >= sensors_by_x[j][0][1]
                        and sensors_by_y[k][0][0] <= sensors_by_x[l][0][0]
                    ):
                        inner_ineq_candidate_combs.append(
                            [
                                [
                                    [-1, -1],
                                    [
                                        -sensors_by_y[k][1]
                                        - 1
                                        - sensors_by_y[k][0][0]
                                        - sensors_by_y[k][0][1]
                                    ],
                                ],
                                [
                                    [-1, +1],
                                    [
                                        -sensors_by_x[j][1]
                                        - 1
                                        - sensors_by_x[j][0][0]
                                        + sensors_by_x[j][0][1]
                                    ],
                                ],
                                [
                                    [+1, -1],
                                    [
                                        -sensors_by_x[i][1]
                                        - 1
                                        + sensors_by_x[i][0][0]
                                        - sensors_by_x[i][0][1]
                                    ],
                                ],
                                [
                                    [+1, +1],
                                    [
                                        -sensors_by_y[l][1]
                                        - 1
                                        + sensors_by_y[l][0][0]
                                        + sensors_by_y[l][0][1]
                                    ],
                                ],
                            ]
                        )
                        inner_candidate_combs.append((k, j, i, l))
                    elif (
                        sensors_by_x[i][0][1] <= sensors_by_x[j][0][1]
                        and sensors_by_y[k][0][0] >= sensors_by_x[l][0][0]
                    ):
                        inner_ineq_candidate_combs.append(
                            [
                                [
                                    [-1, -1],
                                    [
                                        -sensors_by_x[i][1]
                                        - 1
                                        - sensors_by_x[i][0][0]
                                        - sensors_by_x[i][0][1]
                                    ],
                                ],
                                [
                                    [-1, +1],
                                    [
                                        -sensors_by_y[l][1]
                                        - 1
                                        - sensors_by_y[l][0][0]
                                        + sensors_by_y[l][0][1]
                                    ],
                                ],
                                [
                                    [+1, -1],
                                    [
                                        -sensors_by_y[k][1]
                                        - 1
                                        + sensors_by_y[k][0][0]
                                        - sensors_by_y[k][0][1]
                                    ],
                                ],
                                [
                                    [+1, +1],
                                    [
                                        -sensors_by_x[j][1]
                                        - 1
                                        + sensors_by_x[j][0][0]
                                        + sensors_by_x[j][0][1]
                                    ],
                                ],
                            ]
                        )
                        inner_candidate_combs.append((i, l, k, j))

In [17]:
inner_candidates = list()
C = np.array([4000000, 1])
bounds = ((0, 4000000), (0, 4000000))
for comb in inner_ineq_candidate_combs:
    A = list()
    b = list()
    for cond in comb:
        A.append(cond[0])
        b.append(cond[1])
    res_dict = linprog(c=C, A_ub=np.array(A), b_ub=b, bounds=bounds, method="simplex")
    if res_dict["success"]:
        inner_candidates.append((res_dict["x"], res_dict["fun"]))

/tmp/ipykernel_10556/2702053182.py:10: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res_dict = linprog(c=C, A_ub=np.array(A), b_ub=b, bounds=bounds, method='simplex')


In [18]:
print(inner_candidates)

[(array([3340224., 3249595.]), 13360899249595.0)]


All candidates for inner solutions can be tested. If a candidate is admissable, it is the unique solution by assumption:

In [19]:
def test_inner_candidates(sensors, inner_candidates):
    for in_cand in inner_candidates:
        admissible = True
        for sensor in sensors:
            if manh(sensor[0], in_cand[0].astype("int")) < sensor[1] + 1:
                admissible = False
                print(f"Candidate {in_cand} inadmissable!")
                break
        if admissible:
            return in_cand
    return False


inner_test_result = test_inner_candidates(sensors, inner_candidates)
if inner_test_result:
    print(
        f"Solution: (x, y)=({int(inner_test_result[0][0])}, {int(inner_test_result[0][1])}) with tuning frequency {int(inner_test_result[1])}."
    )

Solution: (x, y)=(3340224, 3249595) with tuning frequency 13360899249595.


**Afterthought:** It is a bit of an overkill to use LP here. The inequalities conveyed by A and b reduce to 2 orthogonal lines,
and one just needs to test whether their intersection happens to fall inside the $[0, 4000000]$ x $[0, 4000000]$ square.

**PS:** If there is no solution within `inner_candidates`, one can find the solution on the border of the square of interest i.e. `[0, :]`, `[4000000, :]`, `[:, 0]`, `[:, 4000000]` by following the procedure from *Task 1*.

##### Incomputable NAIVE mathematical solution

In [20]:
# C = np.array([0, 0])
# bounds = ((0, 4000000), (0, 4000000))

In [21]:
# ineqs = list()
# for sensor_pos, radius in sensors:
#     sensor_x, sensor_y = sensor_pos
#     ineqs.append([
#     [[-1, -1], [-radius - 1 - sensor_x - sensor_y]],
#     [[-1, +1], [-radius - 1 - sensor_x + sensor_y]],
#     [[+1, -1], [-radius - 1 + sensor_x - sensor_y]],
#     [[+1, +1], [-radius - 1 + sensor_x + sensor_y]]
#     ])

In [22]:
# perms = product(range(4), repeat=len(sensors))
# i = 0
# for perm in perms:
#     if i > 2000:
#         break
#     i += 1
#     A = list()
#     b = list()
#     for sensor_i, i in enumerate(perm):
#         A.append(ineqs[sensor_i][i][0])
#         b.append(ineqs[sensor_i][i][1])
#     res_dict = linprog(c=C, A_ub=np.array(A), b_ub=b, bounds=bounds)
#     if res_dict['success'] and np.all(res_dict['slack'] <= 1):
#         print('Solution found')
#         break